In [1]:
pip install neo4j-driver

    100% |████████████████████████████████| 184kB 1.8MB/s ta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/50/b5/17/5972f7821c48b83217a47da99387d1f2d67903357a529e752c
  Stored in directory: /home/jovyan/.cache/pip/wheels/29/15/1f/a2d381d5186d84b524791cc3eff45e0e01645a8412e20a1658
  Stored in directory: /home/jovyan/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built neo4j-driver neobolt neotime
Note: you may need to restart the kernel to use updated packages.


In [2]:
from neo4j import GraphDatabase, basic_auth
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
import numpy as np

db_location = "bolt://neo4j:7687"
username = "neo4j"
password = "qwerty"

query = """
        MATCH (p:Person)-[sm:SMOKES]->(t:Tobacco), (p)-[lv:LIVES]->(s:State)
        RETURN s.name as state_name, p.gender as gender, p.age as age, t.name as tobacco
        """


def get_dataset(tx):
    db_res = tx.run(query)
    training_data = pd.DataFrame([r.values() for r in db_res], columns=db_res.keys())
    return training_data


def is_tobacco_of(tobacco: str):
    def go(val: str):
        return tobacco == val
    return go


In [3]:
db = GraphDatabase.driver(db_location, auth=basic_auth(username, password))
feature_encoder = OrdinalEncoder()

with db.session() as session:
    df = session.read_transaction(get_dataset)

df.sample(frac=1).head()

,state_name,gender,age,tobacco
11890,North Carolina,Female,29,Cigarette
5435,Kansas,Male,25,Cigarette
17342,Wyoming,Male,23,Cigarette
7848,Maryland,Male,37,Cigar
14783,South Dakota,Male,22,Cigarette


In [4]:
        y_data = df['tobacco'].apply(is_tobacco_of('Cigar'))
        X_data = np.column_stack((feature_encoder.fit_transform(df.filter(items=['state_name', 'gender'])), df['age'].values))
X_data

array([[ 0.,  0., 27.],
       [ 0.,  0., 29.],
       [ 0.,  0., 26.],
       ...,
       [50.,  1., 58.],
       [50.,  1., 52.],
       [50.,  1., 56.]])

In [5]:
        X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.10, random_state=42)
        logistic_model = LogisticRegression(solver='lbfgs')
        logistic_model.fit(X_train, y_train)
        y_pred = logistic_model.predict(X_test)
        res = len(y_pred[y_pred == y_test]) / len(y_pred)
res

0.8774815655133296

In [6]:
def predict_logistic(model: LogisticRegression, encoder: OrdinalEncoder, df: pd.DataFrame):
    return pd.DataFrame(columns=["does person smoke specified tobacco"],
                        data=model.predict(np.column_stack((encoder.transform(df[['state_name', 'gender']]), df['age'].values))))

In [7]:
predict_logistic(logistic_model, feature_encoder, pd.DataFrame({'state_name': ['Alabama', 'Georgia'],
                                                                'gender': ['Female', 'Male'],
                                                                'age': [33, 55]
                                                                }))

,does person smoke specified tobacco
0,False
1,True
